# Utah Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [1]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [3]:
# state - UPDATE EACH TIME
state = 'utah'

In [ ]:
# review of robots.txt - no restrictions

In [ ]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

## Scrape Links 
`Beautiful Soup`

### Test on one page

In [4]:
link = 'https://governor.utah.gov/news/page/2/'
server_response = requests.get(link, header_list)
server_response

<Response [200]>

In [13]:
soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
results = soup.find('div', class_ = 'posts newsPosts').find_all('div', class_='post newsPost')
# results

title = [i.find('h2').text for i in results]
link = [i.find('a')['href'] for i in results]
date = [i.find('span', class_='postDate').text for i in results]

['Gov. Spencer Cox signs six bills in the 2024 General Legislative Session',
 'Gov. Cox, President Adams, Speaker Schultz issue statement on NHL',
 'Gov. Spencer J. Cox and Lt. Gov. Deidre Henderson: Schedule for Jan. 29- Feb. 2, 2024',
 'Gov. Spencer J. Cox and Lt. Gov. Deidre Henderson: Schedule for Jan. 22-26, 2024',
 'Gov. Spencer Cox signs first bill of the 2024 General Legislative Session',
 'Gov. Cox delivers his 2024 State of the State address',
 'Gov. Spencer J. Cox and Lt. Gov. Deidre Henderson: Schedule for Jan. 15-19, 2024',
 'Gov. Cox encourages educators to remove cell phones during class time',
 'Gov. Spencer J. Cox and Lt. Gov. Deidre Henderson: Schedule for Jan. 8-12, 2024',
 'Gov. Spencer J. Cox and Lt. Gov. Deidre Henderson: Schedule for Jan. 1-5, 2024']

['https://governor.utah.gov/2024/01/30/gov-spencer-cox-signs-six-bills-in-the-2024-general-legislative-session/',
 'https://governor.utah.gov/2024/01/29/gov-cox-president-adams-speaker-schultz-issue-statement-on-nhl/',
 'https://governor.utah.gov/2024/01/29/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-29-feb-2-2024/',
 'https://governor.utah.gov/2024/01/22/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-22-26-2024/',
 'https://governor.utah.gov/2024/01/19/gov-spencer-cox-signs-first-bill-of-the-2024-general-legislative-session/',
 'https://governor.utah.gov/2024/01/18/gov-cox-delivers-his-2024-state-of-the-state-address/',
 'https://governor.utah.gov/2024/01/15/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-15-19-2024/',
 'https://governor.utah.gov/2024/01/10/gov-cox-encourages-educators-to-remove-cell-phones-during-class-time/',
 'https://governor.utah.gov/2024/01/08/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-8-12-20

['January 30, 2024',
 'January 29, 2024',
 'January 29, 2024',
 'January 22, 2024',
 'January 19, 2024',
 'January 18, 2024',
 'January 15, 2024',
 'January 10, 2024',
 'January 8, 2024',
 'December 31, 2023']

In [14]:
# define webpage scraping 
def scrape_utah_links(website, headers):
    server_response = requests.get(website, headers)
    soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
    results = soup.find('div', class_ = 'posts newsPosts').find_all('div', class_='post newsPost')
    results_list = [{'title': i.find('h2').text,
                 'date': i.find('span', class_='postDate').text, 
                'link': i.find('a')['href']} for i in results]
    df = pd.DataFrame.from_dict(results_list)
    df['content'] = np.nan
    return df

### Scrape all links

In [15]:
# try to scrape the first set of pages
# manual review for 2023 - I want pages 2-16, give or take a few links on first and last pages
df_list=[]

for i in range(2,17):
    try: 
        df = scrape_utah_links(f'https://governor.utah.gov/news/page/{i}/', 
                                  header_list)
        df['page_scraped'] = i 
        df_list.append(df)
    except Exception as e:
        print(f'Error: {e} at page {i}')

In [18]:
# combine into one df
combined = pd.concat(df_list, ignore_index=True)
combined.head()
combined.info()

,title,date,link,content,page_scraped
0,Gov. Spencer Cox signs six bills in the 2024 G...,"January 30, 2024",https://governor.utah.gov/2024/01/30/gov-spenc...,NaN,2
1,"Gov. Cox, President Adams, Speaker Schultz iss...","January 29, 2024",https://governor.utah.gov/2024/01/29/gov-cox-p...,NaN,2
2,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,"January 29, 2024",https://governor.utah.gov/2024/01/29/gov-spenc...,NaN,2
3,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,"January 22, 2024",https://governor.utah.gov/2024/01/22/gov-spenc...,NaN,2
4,Gov. Spencer Cox signs first bill of the 2024 ...,"January 19, 2024",https://governor.utah.gov/2024/01/19/gov-spenc...,NaN,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         150 non-null    object 
 1   date          150 non-null    object 
 2   link          150 non-null    object 
 3   content       0 non-null      float64
 4   page_scraped  150 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 6.0+ KB


In [19]:
# export
combined.to_csv(f'{state}_links.csv', index = False)

In [20]:
test = pd.read_csv(f'{state}_links.csv')
test.head()

,title,date,link,content,page_scraped
0,Gov. Spencer Cox signs six bills in the 2024 G...,"January 30, 2024",https://governor.utah.gov/2024/01/30/gov-spenc...,NaN,2
1,"Gov. Cox, President Adams, Speaker Schultz iss...","January 29, 2024",https://governor.utah.gov/2024/01/29/gov-cox-p...,NaN,2
2,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,"January 29, 2024",https://governor.utah.gov/2024/01/29/gov-spenc...,NaN,2
3,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,"January 22, 2024",https://governor.utah.gov/2024/01/22/gov-spenc...,NaN,2
4,Gov. Spencer Cox signs first bill of the 2024 ...,"January 19, 2024",https://governor.utah.gov/2024/01/19/gov-spenc...,NaN,2


## Scrape content from links
`Beautiful Soup`

### Test with one link

In [21]:
# test with one
link = 'https://governor.utah.gov/2023/01/17/news-release-gov-spencer-cox-appoints-jason-nelson-to-2nd-district-court/'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [23]:
# test with one contd 
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('article', class_ = 'post wrapper').find_all('p')
string_result = [str(i.text) for i in result]
string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
result_merged = " ".join(string_result_clean)
result_merged

'NEWS RELEASE Jan. 17, 2023 Contact: Emma Williams Office of the Governor (385) 303-4383, ewilliams@utah.gov Gov. Spencer Cox appoints Jason Nelson to 2nd District Court SALT LAKE CITY (Jan. 17, 2023) – Utah Gov. Spencer J. Cox has appointed Jason Nelson to Utah’s 2nd District Court, filling the vacancy left by Judge David Connors’ retirement. Judicial appointments are subject to confirmation by the Utah Senate. “I am pleased that Jason Nelson is willing to serve Utahns in this new capacity,” Gov. Cox said. “His professionalism and love of our state will serve the community well.” Nelson has served as the Deputy Davis County Attorney since 2008. He has also served as a prosecutor for West Bountiful City and North Salt Lake City. Prior to that, he served as a law clerk in the Utah Second Judicial District in Davis County as well as the Utah Attorney General’s Office Criminal Justice Division. “I am grateful for and humbled by the confidence Gov. Cox has placed in me with this appointmen

In [ ]:
# define link scraping 
def get_utah_content(link, df, header_list): 
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('article', class_ = 'post wrapper').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
                string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
                result_merged = " ".join(string_result_clean)
                
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [25]:
# test on 5 to see if it works
# links_2023.content = np.nan

for i in combined.link[0:6]:
     get_utah_content(i, combined, header_list)
    
combined.head()

'success: content added from https://governor.utah.gov/2024/01/30/gov-spencer-cox-signs-six-bills-in-the-2024-general-legislative-session/'

'success: content added from https://governor.utah.gov/2024/01/29/gov-cox-president-adams-speaker-schultz-issue-statement-on-nhl/'

'success: content added from https://governor.utah.gov/2024/01/29/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-29-feb-2-2024/'

'success: content added from https://governor.utah.gov/2024/01/22/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-22-26-2024/'

'success: content added from https://governor.utah.gov/2024/01/19/gov-spencer-cox-signs-first-bill-of-the-2024-general-legislative-session/'

'success: content added from https://governor.utah.gov/2024/01/18/gov-cox-delivers-his-2024-state-of-the-state-address/'

,title,date,link,content,page_scraped
0,Gov. Spencer Cox signs six bills in the 2024 G...,"January 30, 2024",https://governor.utah.gov/2024/01/30/gov-spenc...,"NEWS RELEASE Jan. 30, 2024 Contact: Emma Willi...",2
1,"Gov. Cox, President Adams, Speaker Schultz iss...","January 29, 2024",https://governor.utah.gov/2024/01/29/gov-cox-p...,"NEWS RELEASE Jan. 29, 2024 Contacts: Emma Will...",2
2,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,"January 29, 2024",https://governor.utah.gov/2024/01/29/gov-spenc...,**Events labeled Media Access indicate that an...,2
3,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,"January 22, 2024",https://governor.utah.gov/2024/01/22/gov-spenc...,**Events labeled Media Access indicate that an...,2
4,Gov. Spencer Cox signs first bill of the 2024 ...,"January 19, 2024",https://governor.utah.gov/2024/01/19/gov-spenc...,"NEWS RELEASE Jan. 19, 2024 Contact: Emma Willi...",2


### Scrape content from all links

In [26]:
for i in combined.link:
     get_utah_content(i, combined, header_list)

'content already existing for https://governor.utah.gov/2024/01/30/gov-spencer-cox-signs-six-bills-in-the-2024-general-legislative-session/'

'content already existing for https://governor.utah.gov/2024/01/29/gov-cox-president-adams-speaker-schultz-issue-statement-on-nhl/'

'content already existing for https://governor.utah.gov/2024/01/29/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-29-feb-2-2024/'

'content already existing for https://governor.utah.gov/2024/01/22/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-22-26-2024/'

'content already existing for https://governor.utah.gov/2024/01/19/gov-spencer-cox-signs-first-bill-of-the-2024-general-legislative-session/'

...

'success: content added from https://governor.utah.gov/2023/01/11/gov-spencer-cox-appoints-judge-amy-j-oliver-to-utah-court-of-appeals/'

'success: content added from https://governor.utah.gov/2023/01/10/media-release-gov-cox-hosts-social-media-and-youth-mental-health-symposium/'

'success: content added from https://governor.utah.gov/2023/01/09/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-9-jan-13-2023/'

'success: content added from https://governor.utah.gov/2023/01/05/icymi-what-does-it-mean-to-be-fiscally-responsible-in-utah/'

'success: content added from https://governor.utah.gov/2023/01/04/gov-spencer-j-cox-and-lt-gov-deidre-henderson-schedule-for-jan-2-jan-6-2023/'

## Validate and Clean

In [27]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,date,link,content,page_scraped


In [28]:
# convert to date-time format
combined['date_clean'] = pd.to_datetime(combined['date'])

combined_2023 = combined.drop(columns=['date']).copy()

In [29]:
combined_2023.head()
combined_2023.info()

,title,link,content,page_scraped,date_clean
0,Gov. Spencer Cox signs six bills in the 2024 G...,https://governor.utah.gov/2024/01/30/gov-spenc...,"NEWS RELEASE Jan. 30, 2024 Contact: Emma Willi...",2,2024-01-30
1,"Gov. Cox, President Adams, Speaker Schultz iss...",https://governor.utah.gov/2024/01/29/gov-cox-p...,"NEWS RELEASE Jan. 29, 2024 Contacts: Emma Will...",2,2024-01-29
2,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,https://governor.utah.gov/2024/01/29/gov-spenc...,**Events labeled Media Access indicate that an...,2,2024-01-29
3,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,https://governor.utah.gov/2024/01/22/gov-spenc...,**Events labeled Media Access indicate that an...,2,2024-01-22
4,Gov. Spencer Cox signs first bill of the 2024 ...,https://governor.utah.gov/2024/01/19/gov-spenc...,"NEWS RELEASE Jan. 19, 2024 Contact: Emma Willi...",2,2024-01-19


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         150 non-null    object        
 1   link          150 non-null    object        
 2   content       150 non-null    object        
 3   page_scraped  150 non-null    int64         
 4   date_clean    150 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 6.0+ KB


In [30]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 150 documents from utah for 2023


## Export

In [31]:
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [32]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,title,link,content,page_scraped,date_clean
0,Gov. Spencer Cox signs six bills in the 2024 G...,https://governor.utah.gov/2024/01/30/gov-spenc...,"NEWS RELEASE Jan. 30, 2024 Contact: Emma Willi...",2,2024-01-30
1,"Gov. Cox, President Adams, Speaker Schultz iss...",https://governor.utah.gov/2024/01/29/gov-cox-p...,"NEWS RELEASE Jan. 29, 2024 Contacts: Emma Will...",2,2024-01-29
2,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,https://governor.utah.gov/2024/01/29/gov-spenc...,**Events labeled Media Access indicate that an...,2,2024-01-29
3,Gov. Spencer J. Cox and Lt. Gov. Deidre Hender...,https://governor.utah.gov/2024/01/22/gov-spenc...,**Events labeled Media Access indicate that an...,2,2024-01-22
4,Gov. Spencer Cox signs first bill of the 2024 ...,https://governor.utah.gov/2024/01/19/gov-spenc...,"NEWS RELEASE Jan. 19, 2024 Contact: Emma Willi...",2,2024-01-19
